In [ ]:
!pip install pytesseract

In [33]:
from PIL import Image
import pytesseract
import openai
import json
import os
import base64
import requests

In [35]:
creds = json.load(open('../work/llm-discovery/chatgpt_creds.json'))
openai.api_key = creds['OPENAI_API_KEY']

# Function to extract text from an image using Tesseract OCR
def extract_text_from_image(image_path):
    img = Image.open(image_path)
    text = pytesseract.image_to_string(img, lang = 'ukr')
    return text

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')


def process_image_with_gpt(image_path):    
    # Getting the base64 string
    base64_image = encode_image(image_path)
    
    headers = {
      "Content-Type": "application/json",
      "Authorization": f"Bearer {openai.api_key}"
    }
    
    payload = {
      "model": "gpt-4o",
      "messages": [
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": "Extract all the phrases from this image as a valid Python list"
            },
            {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"
              }
            }
          ]
        }
      ],
      "max_tokens": 300
    }
    
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    return response

In [29]:
image_paths = os.listdir('images')

In [36]:
extracted = []
for image_path in image_paths:
    extracted.append(process_image_with_gpt('images/'+image_path))

    
    # # Optionally process the text with GPT
    # response_text = process_text_with_gpt(extracted_text)
    # print(f"GPT-Processed Text:\n{response_text}\n")

In [69]:
test = extracted[0].json()['choices'][0]['message']['content']

'phrases = [\n    "Красива",\n    "Цілеспрямованість, наполегливість, розвиток, внутрішня сила та краса",\n    "Цікава дівчина",\n    "PhD😯",\n    "Енергія, професійність відкритість, доброзичливість, проф.ріст, допомога",\n    "Трудоголік",\n    "Професіоналізм, цілеспрямованість, драйв"\n]'

In [77]:
import ast

def extract_phrases(text):
    print(text)
    text = text.strip('```').strip('python').strip()
    text = text.split('=', 1)[1].strip()
    return ast.literal_eval(text)
    

In [78]:
phrases = [extract_phrases(res.json()['choices'][0]['message']['content']) for res in extracted]

```python
phrases = [
    "Красива",
    "Цілеспрямованість, наполегливість, розвиток, внутрішня сила та краса",
    "Цікава дівчина",
    "PhD😯",
    "Енергія, професійність відкритість, доброзичливість, проф.ріст, допомога",
    "Трудоголік",
    "Професіоналізм, цілеспрямованість, драйв"
]
```
```python
[
    "Аналіз",
    "успіх!!",
    "Сучасна, програмування, передбачення, драйв, Poland, працьовитість",
    "Захопленість",
    "Сміливість і успіх",
    "Смак до життя",
    "Красива",
    "Цілеспрямованість, наполегливість, розвиток, внутрішня сила та краса"
]
```


IndexError: list index out of range